# Molecular Imaging: A Hands-on Tutorial
Welcome to the hands-on tutorial on Molecular Imaging based on Open Access Datasets. This notebook will guide you through the installation and use of several important neuroimaging software tools, including FreeSurfer and FSL, using Python and Jupyter notebooks in Google Colab. We will use open access datasets to demonstrate real-world neuroimaging data preprocessing of PET data.

## Objectives
- Learn how to install necessary neuroimaging tools in Colab.
- Understand how to process neuroimaging data using these tools.
- Apply this knowledge to open access datasets for practical understanding.

### Prerequisites
Before starting, ensure you have a Google account and are familiar with basic Python programming and Jupyter notebook operations.

## Collect the Repository and Install all Dependencies
This may take some time.

In [4]:
!if [ -d "outreach" ]; then echo "Repo already cloned." && cd outreach && git pull && cd ..; else echo "Collecting Outreach Repository" && git clone https://github.com/openneuropet/outreach.git; fi
from os.path import basename
from os import getcwd
if basename(getcwd()) == 'OHBM2024':
    pass
else:
    %cd outreach/OHBM2024/

base_dir = getcwd()
!apt-get install pigz tree

Repo already cloned.
Already up to date.
/content/outreach/OHBM2024
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pigz is already the newest version (2.6-1).
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


## Install FreeSurfer
Note, this is only necessary if you want to actually run the pipelines below


In [7]:
# Download freesurfer 7.4.1
!wget https://surfer.nmr.mgh.harvard.edu/pub/dist/freesurfer/7.4.1/freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz
!if [ -f freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz ]; then time pigz -dc freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz | tar xf -; fi
import os, sys
# append freesurfer to system path
freesurfer_path = os.path.join(os.getcwd(), 'freesurfer/bin/')
os.getenv('PATH')
if freesurfer_path not in os.environ['PATH']:
  os.environ['PATH'] = freesurfer_path + ':' + os.getenv('PATH')
freesurfer_home = os.environ['FREESURFER_HOME'] = os.path.join(os.getcwd(), 'freesurfer')
subjects_dir = os.environ['SUBJECTS_DIR'] = os.path.join(os.getcwd(), 'freesurfer', 'subjects')

def set_global_variables():
    global freesurfer_home
    global subjects_dir

set_global_variables()

# copy license file into freesurfer home
!cp license.txt $FREESURFER_HOME/
!echo $PATH
!mri_coreg --version

os.environ['FREESURFER_HOME'] = freesurfer_home
os.environ['SUBJECTS_DIR'] = subjects_dir


/content/outreach/OHBM2024/freesurfer/bin/:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
mri_coreg freesurfer 7.4.1


## Install PETPrep robust head motion correction workflow (petprep_hmc)
Note, this is only necessary if you want to actually run the pipelines below


In [12]:
!if [ -d "petprep_hmc" ]; then pushd petprep_hmc && git pull; else git clone https://github.com/mnoergaard/petprep_hmc.git; fi
%cd petprep_hmc
!pip install -e .

/content/outreach/OHBM2024/code/petprep_hmc /content/outreach/OHBM2024/code
Already up to date.


## Download PET-BIDS data for preprocessing (full dataset can be found here: https://openneuro.org/datasets/ds004869/versions/1.0.1)
Note, this is only necessary if you want to actually run the pipelines below


In [25]:
data_path = os.path.join(base_dir, 'data')
os.makedirs(data_path, exist_ok=True)
%cd data_path
!wget https://www.dropbox.com/scl/fo/cfm8wc658jy49oazxzukq/AOPbFDHjiBp9qqXggffMrNk?rlkey=r5t22nkokqqk1bd5wamqy93wj&st=qbzvavav&dl=0
!if [ -f AOPbFDHjiBp9qqXggffMrNk?rlkey=r5t22nkokqqk1bd5wamqy93wj ]; then unzip -o AOPbFDHjiBp9qqXggffMrNk?rlkey=r5t22nkokqqk1bd5wamqy93wj; fi

Archive:  AOPbFDHjiBp9qqXggffMrNk?rlkey=r5t22nkokqqk1bd5wamqy93wj
mapname:  conversion of  failed
 extracting: README                  
   creating: sub-01/
 extracting: LICENSE                 
   creating: derivatives/
   creating: sub-01/anat/
 extracting: participants.tsv        
 extracting: participants.json       
   creating: sub-01/ses-baseline/
   creating: derivatives/smriprep/
   creating: derivatives/freesurfer/
   creating: sub-01/ses-baseline/pet/
 extracting: dataset_description.json  
   creating: derivatives/smriprep/sub-01/
 extracting: sub-01/anat/sub-01_T1w.json  
   creating: derivatives/freesurfer/sub-01/
 extracting: sub-01/anat/sub-01_T1w.nii.gz  
 extracting: derivatives/smriprep/sub-01.html  
   creating: derivatives/freesurfer/fsaverage/
   creating: derivatives/smriprep/sub-01/anat/
   creating: derivatives/freesurfer/sub-01/mri/
   creating: derivatives/freesurfer/sub-01/surf/
   creating: derivatives/smriprep/sub-01/figures/
   creating: derivatives/frees

## Install FSL
Note, this is only necessary if you want to actually run the pipelines below


In [31]:
%cd base_dir

import subprocess
import os

# Define a function to run shell commands and handle errors
def run_command(command, input=None, cwd=None):
    try:
        result = subprocess.run(command, input=input, text=True, capture_output=True, shell=True, cwd=cwd)
        print(result.stdout)
        if result.stderr:
            print("Error:", result.stderr)
    except Exception as e:
        print("An error occurred:", e)

# Download FSL installer script
print("Downloading FSL installer...")
run_command("wget -O fslinstaller.py https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py")

# Run the FSL installer with non-interactive flags
print("Running FSL installer non-interactively...")
installer_command = f"python fslinstaller.py --dest={base_dir}/fsl"
run_command(installer_command)

# Set environment variables for the session
fsl_dir = f"{base_dir}/fsl"  # Adjust this path according to your installation
os.environ['FSLDIR'] = fsl_dir
os.environ['PATH'] += f":{fsl_dir}/bin"
os.environ['FSLOUTPUTTYPE'] = "NIFTI_GZ"

# Source FSL configuration script (this step might not fully work in Jupyter as mentioned before)
print("Configuring FSL environment...")
run_command(f"source {fsl_dir}/etc/fslconf/fsl.sh")

print("FSL Installation and configuration complete.")


Error: --2024-06-13 18:39:39--  https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
Resolving fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)... 129.67.248.65
Connecting to fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)|129.67.248.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74517 (73K) [application/octet-stream]
Saving to: ‘fslinstaller.py’

     0K .......... .......... .......... .......... .......... 68%  307K 0s
    50K .......... .......... ..                              100%  255M=0.2s

2024-06-13 18:39:40 (447 KB/s) - ‘fslinstaller.py’ saved [74517/74517]


Running FSL installer non-interactively...
FSL installer version: 3.2.0
Press CTRL+C at any time to cancel installation
Running the installer script as root user is discouraged! You should run this script as a regular user - you will be asked for your administrator password if required.
Installation log file: /tmp/fslinstaller_zfhpsa13.log

FSL installer version: 3.12.0
Press CTRL+C at any time to canc

## Run the petprep_hmc workflow


In [33]:
petprep_hmc_path = os.path.join(base_dir, 'petprep_hmc')
%cd $petprep_hmc_path

/content/outreach/OHBM2024/code/petprep_hmc


In [ ]:
!python3 run.py --bids_dir {base_dir}/data --output_dir {base_dir}/data/derivatives/petprep_hmc --n_procs 4

FSL is installed at: /content/outreach/OHBM2024/fsl
240613-18:56:19,595 nipype.workflow INFO:
	 Workflow petprep_hmc_wf settings: ['check', 'execution', 'logging', 'monitoring']
240613-18:56:19,618 nipype.workflow INFO:
	 Running in parallel.
240613-18:56:19,641 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 11.41/11.41, Free processors: 4/4.
240613-18:56:19,806 nipype.workflow INFO:
	 [Node] Setting-up "petprep_hmc_wf.subject_01_wf.select_files" in "/content/outreach/OHBM2024/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/select_files".
240613-18:56:19,813 nipype.workflow INFO:
	 [Node] Executing "select_files" <nipype.interfaces.io.SelectFiles>
240613-18:56:19,824 nipype.workflow INFO:
	 [Node] Finished "select_files", elapsed time 0.003352s.
240613-18:56:21,634 nipype.workflow INFO:
	 [Job 0] Completed (petprep_hmc_wf.subject_01_wf.select_files).
240613-18:56:21,637 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, an